In [207]:
import pandas as pd

In [208]:
df = pd.read_csv('../data/eng-hin/eng-hin-tabfile.txt', sep='\t')

In [209]:
df.head(10)

,##name,eng-hin.index
0,##sourceLang,English
1,##targetLang,Hindi
2,00databasealphabet,12abcdefghijklmnopqrstuvwxyz<br />
3,00databasedictfmt1121,00-database-dictfmt-1.12.1<br />
4,00databaseinfo,English-Hindi FreeDict Dictionary<br /><br />M...
5,00databaseshort,English-Hindi FreeDict Dictionary ver. 1.6<br />
6,00databaseurl,unknown<br />
7,00databaseutf8,<br />
8,a,"a /ˈeɪ/ <Det><br />1. एक<br /> ""I bought ..."
9,a feather in ones cap,A feather in one's cap /ɐ fˈɛðəɹ ɪn wˈɒnz kˈap...


In [210]:
dict_df = df[8:].copy()
print(len(dict_df))
display(dict_df.head())

22871


,##name,eng-hin.index
8,a,"a /ˈeɪ/ <Det><br />1. एक<br /> ""I bought ..."
9,a feather in ones cap,A feather in one's cap /ɐ fˈɛðəɹ ɪn wˈɒnz kˈap...
10,a few,"a few /ɐ fjˈuː/ <Det><br />1. कुछ<br /> ""..."
11,a fight to the finish,a fight to the finish /ɐ fˈaɪt tə ðə fˈɪnɪʃ/ <...
12,a fighting chance,A fighting chance /ɐ fˈaɪtɪŋ tʃˈans/ <IDM><br ...


In [211]:
dict_df.rename({'##name': 'word/phrase', 'eng-hin.index': 'definition'}, axis=1, inplace=True)

In [212]:
dict_df.head()

,word/phrase,definition
8,a,"a /ˈeɪ/ <Det><br />1. एक<br /> ""I bought ..."
9,a feather in ones cap,A feather in one's cap /ɐ fˈɛðəɹ ɪn wˈɒnz kˈap...
10,a few,"a few /ɐ fjˈuː/ <Det><br />1. कुछ<br /> ""..."
11,a fight to the finish,a fight to the finish /ɐ fˈaɪt tə ðə fˈɪnɪʃ/ <...
12,a fighting chance,A fighting chance /ɐ fˈaɪtɪŋ tʃˈans/ <IDM><br ...


In [213]:
word = ''
data = ''
count = 1
for row in dict_df.itertuples():
    index = row[0]
    word = row[1]
    data = row[2]
    print(index, word, data)
    if count % 10 == 0:
        break
    count += 1
print(word, data)

8 a a /ˈeɪ/ <Det><br />1. एक<br />      "I bought a pen."<br />
9 a feather in ones cap A feather in one's cap /ɐ fˈɛðəɹ ɪn wˈɒnz kˈap/ <IDM><br />1. कठिन~कार्य~को~करना<br />      "Getting this award is like a feather in his cap."<br />
10 a few a few /ɐ fjˈuː/ <Det><br />1. कुछ<br />      "Only a few students of this class can expect to get first division in their examination results."<br />2.  In the party only a few persons were known to me<br /><br /><br /><hr><br /><br />a few /ɐ fjˈuː/ <Adv><br />1. कुछ<br />      "Please give me a few more details of this project."<br />
11 a fight to the finish a fight to the finish /ɐ fˈaɪt tə ðə fˈɪnɪʃ/ <IDM><br />1. निर्णायक~लड़ाई<br />      "It was a fight to finish between the two chess players. "<br />
12 a fighting chance A fighting chance /ɐ fˈaɪtɪŋ tʃˈans/ <IDM><br />1. कार्य~की~सफलता~की~थोड़ी~आशा<br />      "This leader has only a fighting chance of success in the election."<br />
13 a fly on the wall a fly on the wall /ɐ flˈaɪ ɒnðə w

In [214]:
row = dict_df.iloc[110]
row

word/phrase                                                abuse
definition     abuse /ɐbjˈuːs/ <N><br />1. दुर्व्यवहार<br /> ...
Name: 118, dtype: object

In [215]:
import json
import re


def parse_definition(definition_text):
    # Split input by parts of speech based on <N> and <VT> markers
    # parts = re.split(r'<([NVIT]+)>', definition_text)
    parts = re.split(r'<([NVVIVTAdjAdvPronPrepConjDetNumInterjIDMPhrvAbbrSlExclAuxPlArt]+)>', definition_text)

    # Extract the word and pronunciation
    word_section = parts[0].strip()
    # pattern_word_section = r"^([\w\s]+)\s/([^/]+)/"
    # pattern_word_section = r"^([\w\s]+)\s/([ˈˌaæbddʒefghijklmnŋoɔprstt̬tʃuvwxɐɑɒɑ̃ɚəɜɝɛɪʃʊʌʒθð\s]+)/"
    # pattern_word_section = r"([\w\d\s'.]+)\/([aäɑɒæbḇβcčɔɕçdḏḍðeəɚɛɝfgḡhʰḥḫẖiɪỉɨjʲǰkḳḵlḷɬɫmnŋṇɲɴoŏɸθpp̅þqrɹɾʀʁṛsšśṣʃtṭṯʨtʂuʊŭüvʌɣwʍxχyʸʎzẓžʒ’‘ʔʕ\s]+)\/"
    pattern_word_section = r"([\w\d\s'.\-()~]+)\/(.+)\/"
    word, pronunciation = re.match(
        pattern_word_section, word_section).groups()

    word = word.strip()

    # Initialize result structure
    result = {
        "word": word,
        "pronunciation": pronunciation,
        "entries": []
    }

    # Process each POS section
    for i in range(1, len(parts) - 1, 2):
        part_of_speech = parts[i].strip()
        raw_definitions = parts[i + 1].strip()

        # Map POS markers to human-readable forms
        pos_map = {
            "N": "Noun",
            "V": "Verb",
            "VI": "Verb (Intransitive)",
            "VT": "Verb (Transitive)",
            "Adj": "Adjective",
            "Adv": "Adverb",
            "Pron": "Pronoun",
            "Prep": "Preposition",
            "Conj": "Conjunction",
            "Det": "Determiner",
            "Num": "Numeral",
            "Interj": "Interjection",
            "IDM": "Idiom",
            "Phrv": "Phrasal Verb",
            "Abbr": "Abbreviation",
            "Sl": "Slang",
            "Excl": "Exclamation",
            "Aux": "Auxiliary Verb",
            "Pl": "Plural",
            "Art": "Article"
        }
        pos_name = pos_map.get(part_of_speech, part_of_speech)

        # Parse definitions
        definitions = []
        for entry in re.split(r'\d+\.\s', raw_definitions)[1:]:
            match = re.match(r'([^<]+)<br\s*/>\s*"([^"]+)"', entry, re.DOTALL)
            if match:
                hindi, example = match.groups()
                definitions.append({
                    "hindi": hindi.strip().replace('~', ' '),
                    "example": example.strip()
                })

        # Add parsed data to the entries
        result["entries"].append({
            "part_of_speech": pos_name,
            "definitions": definitions
        })

    return result

In [216]:
print(data)
parsed_data = parse_definition(data)
# print(row.definition)
# parsed_data = parse_definition(row.definition)
print(parsed_data)

A level /ɐ lˈɛvəl/ <N><br />1. उच्च स्तरीय<br />      "She will take the A level exams this year."<br />
{'word': 'A level', 'pronunciation': 'ɐ lˈɛvəl', 'entries': [{'part_of_speech': 'Noun', 'definitions': [{'hindi': 'उच्च स्तरीय', 'example': 'She will take the A level exams this year.'}]}]}


In [217]:
row = dict_df.iloc[113]
row

word/phrase                                              abysmal
definition     abysmal /ɐbˈɪsməl/ <Adj><br />1. बहुत~खराब<br ...
Name: 121, dtype: object

In [218]:
parsed_data = parse_definition(row.definition)
print(parsed_data)

{'word': 'abysmal', 'pronunciation': 'ɐbˈɪsməl', 'entries': [{'part_of_speech': 'Adjective', 'definitions': [{'hindi': 'बहुत खराब', 'example': 'His habits are abysmal.'}]}]}


In [219]:
all_words = {}

word = ''
unparsed_words = []
for row in dict_df.itertuples():
    index = row[0]
    word = row[1]
    data = row[2]
    try:
        parsed_data = parse_definition(data)
        all_words[word] = parsed_data
    except Exception as e:
        # print(e)
        parts = re.split(r'<([NVVIVTAdjAdvPronPrepConjDetNumInterjIDMPhrvAbbrSlExclAuxPlArt]+)>', data)
        word_section = parts[0].strip()
        print(word_section)
        unparsed_words.append(word)

with open('../data/eng-hin/eng-hin-parsed.json', 'w') as f:
    f.write(json.dumps(all_words))
    print(len(list(all_words.keys())))

with open('../data/eng-hin/eng-hin-unparsed.json', 'w') as f:
    f.write(json.dumps(unparsed_words))
    print(len(unparsed_words))

flag[stone] /flˈaɡ stˈəʊn/
keep in with <a href="bword://infl">infl</a> /kˈiːp ɪn wɪð ˈɪnfəl/
not care\\give a fig /nˌɒt kˈeə bˈakslaʃ ɡˈɪv ɐ fˈɪɡ/
pell-?????? /pˈɛl/
raise (US=rise) /ɹˈeɪz jˌuːˈɛs ˈiːkwəlz ɹˈaɪz/
tu-whit,tu-whoo /tˈuːwˈɪt tˈuːwˈuː/
wellington[boot] /wˈɛlɪŋtən bˈuːt/
22864
7


In [220]:
def reverse_definition(definition):
    pass